In [1]:
from ai.lecture_notes.llama_predict import llama_promt
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from fuzzywuzzy import fuzz

from langdetect import detect
from nltk.stem import WordNetLemmatizer
from pymorphy2 import MorphAnalyzer
import nltk

from ai.transcriptor.Exctract_keywords import get_keywords, combined_stop_words
from ai.transcriptor.STT import speech2text
from ai.transcriptor.timecodes_and_full_text import timecodes_and_full_text

lang = 'Русский'

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nikolaibabuhin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
nltk.download('wordnet')

model_name = "facebook/nllb-200-distilled-600M"
model_t = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/nikolaibabuhin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
d_langs_t = {'Английский': 'eng_Latn',
             'Датский': 'dan_Latn',
             'Испанский': 'spa_Latn',
             'Итальянский': 'ita_Latn',
             'Китайский': 'zho_Hans',
             'Немецкий': 'deu_Latn',
             'Польский': 'pol_Latn',
             'Португальский': 'por_Latn',
             'Турецкий': 'tur_Latn',
             'Французский': 'fra_Latn',
             'Чешский': 'ces_Latn',
             'Русский': 'rus_Cyrl'
             # 'Японский': 'jpn_Jpan',
             }

In [4]:
def lemmatize_word(word):
    # Определяем язык слова
    language = detect(word)

    if language == "en":
        lemmatizer = WordNetLemmatizer()
        return lemmatizer.lemmatize(word)
    elif language == "ru":
        morph = MorphAnalyzer()
        return morph.parse(word)[0].normal_form
    else:
        return word  # Если язык не определен или не поддерживается

In [5]:

# Define a function to remove similar words
def remove_similar_words(keywords):
    import pymorphy2
    morph = pymorphy2.MorphAnalyzer()
    unique_keywords = []
    for keyword in keywords:
        is_similar = False
        for unique_keyword in unique_keywords:
            if fuzz.ratio(keyword[0], unique_keyword[0]) > 70:
                is_similar = True
                break
        if not is_similar:
            #teg = morph.parse(keyword[0])[0]
            unique_keywords.append((lemmatize_word(keyword[0]), keyword[1]))
    return unique_keywords

In [6]:
def translator(text, target_language):
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model_t.generate(**inputs, forced_bos_token_id=tokenizer.lang_code_to_id[target_language])
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)


In [7]:
def sound_to_text(audio_file):
    lang = 'Русский'
    conspect = timecodes_and_full_text(audio_file)
    """
    С тайм кодами разбивка + конспект лекции
    timecode_with_text': [[x['start'], x['end'], x['text']] for x in data['segments']],
        'text': data['text']
    """
    print(conspect)
    text = conspect['text']
    termins = get_keywords(text, stopwords=combined_stop_words, keyphrase_ngram_range=(1, 1), top_n=200)
    # Remove similar words
    termins = remove_similar_words(termins)
    print(termins)
    """[('python', 0.4077), ('программирования', 0.31), ('разработчика', 0.2249)]"""
    out = []
    """for x in termins:
        if len(x[0]) < 499:
            out.append(translator(llama_promt(x[0]), d_langs_t.get(lang, 'rus_Cyrl')))
        else:
            print("превышен порог в 500 символов")"""
    #print(out)
    return out, conspect, termins

In [8]:
import pandas as pd

# Создайте DataFrame с данными для каждой колонки
data = {
    'File': [],
    'Term': []
}

# Создайте DataFrame из словаря данных
df = pd.DataFrame(data)

# Запишите DataFrame в CSV файл
df.to_csv('new_test.csv', index=False)

In [ ]:
import os

# Укажите путь к папке, в которой находятся файлы
folder_path = './new_test/audiofiles'

# Получите список всех файлов в папке
file_list = os.listdir(folder_path)

# Переберите каждый файл в списке
for file_name in file_list:
    # Проверьте, является ли элемент путем к файлу
    if os.path.isfile(os.path.join(folder_path, file_name)):
        # Выведите ссылку на файл
        print(os.path.join(folder_path, file_name))
        out, conspect, termins = sound_to_text(os.path.join(folder_path, file_name))
        termins
        for x in termins:
                # Создайте новую строку, которую хотите добавить
            new_row = {'File': folder_path,
                       'Term': translator(llama_promt(x[0]), d_langs_t.get(lang, 'rus_Cyrl'))}
            df = df.append(new_row, ignore_index=True)

./new_test/audiofiles/audio3.mp3
Аудиофайл разбит на сегменты


/Users/nikolaibabuhin/PycharmProjects/geekbrains-ai-assistant/venv/lib/python3.9/site-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
